### Import all required functions:

In [ ]:
from utilities import *
from convenience_functions import *
from functions import *

### Load files:

In [ ]:
fpath = r'/Users/ryan/Desktop/polo/PLM_data/2023_02_03_Anna/AD004_FullRecovery'
str_filter = 'Orientation'
file_type = '.tif'

arrs, names = load_orientation_files(fpath, str_filter, file_type)

### Preprocess files as requried:

In [ ]:
norm_arrs = prepro_orientation(arrs)

### Display images:

In [ ]:
for i in norm_arrs:
    plt.imshow(i, cmap = 'Greys')
    plt.axis('off')
    plt.show()

### Set step size for binning pixel values:

In [ ]:
step_size = 5

### Set empty lists with the same length as number of files to collect analysis results:

In [ ]:
n_arrs = len(norm_arrs)

bin_pairs = [0] * n_arrs
filt_arrs = [0] * n_arrs
labeled_arrs = [0] * n_arrs

### Loop over all images to analyse:

In [ ]:
counter = 0
for i in norm_arrs:
    
    idx_steps, arr_steps, val_pairs = percentile_pairs(i, step_size)
    filt = filter_arr_vals(i, val_pairs)
    labeled, ct = label_expansion(filt, noise_se_size=5)
    
    bin_pairs[counter] = val_pairs
    filt_arrs[counter] = filt
    labeled_arrs[counter] = labeled
    
    counter += 1

### Display labeled results:

In [ ]:
for i in labeled_arrs:
    plt.imshow(i.sum(axis = 0), cmap = 'Set3')
    plt.axis('off')
    plt.show()

### Set information for saving results:

In [ ]:
## this removes the filetype specification from the end of filename
str_trim = len(file_type)

In [ ]:
## this will put today's date in the directory name of the analysis results
date = datetime.now().isoformat(sep = '_', timespec = 'seconds').split('_')[0]

In [ ]:
## this creates a directory to save the analysis results
save_path = os.path.join(fpath, date + '_analysis_stepsize_' + str(step_size) + '_' + str_filter)
os.mkdir(save_path)

In [ ]:
## choose which results you want to save and how to name them
save_names = ('preprocessed','bins','labeled')
save_files = (norm_arrs, bin_pairs, labeled_arrs)

### Save results:

In [ ]:
## save all results as numpy files
for i in range(len(names)):
    for j in range(len(save_names)):
        np.save(os.path.join(save_path, names[i][:-str_trim] + '_' + save_names[j]), save_files[j][i])

In [ ]:
## save labeled image from analysis results as tiff file     
for i in range(len(names)):
    imsave(os.path.join(save_path, names[i][:-str_trim] + '_labeled.tif'), labeled_arrs[i].astype(np.uint16))